# Permite extraer las imágenes de los diagramas de un documento DOCx o PDF

0) Preparar ambiente e instalar paquetes:

In [ ]:
#@title Instala paquetes 

# para procesar PDFs [https://pymupdf.readthedocs.io/en/latest/]
!pip install PyMuPDF

print("\n")

# para procesar DocX [https://pypi.org/project/docx2python/]
!pip install docx2python

1) Cargar librerías:

In [ ]:
#@title Cargar Librerías
import os

import fitz
from docx2python import docx2python

print("Librerías cargadas.")

2) Montar el Drive:

In [ ]:
#@title Montar Google Drive
# Nota: la primera vez se debe confirmar el uso logueandose en "Google Drive File Stream" y obteniendo código de autentificación.
from google.colab import drive
drive.mount('/content/gdrive', force_remount=True)

3) Lleva a cabo el procesamiento de los documentos:

In [ ]:
#@title Procesa los documentos y graba las imágenes correspondientes

drive_path = '/content/gdrive/My Drive/GEMIS/objDetectionCursogramas/Cursogramas' #@param {type:"string"}
drive_origen_documentos = '/Diagramas-Enunciados/Originales' #@param {type:"string"}
drive_destino_diagramas = '/Diagramas-Enunciados' #@param {type:"string"}
origen_dir_path = drive_path + drive_origen_documentos
destino_dir_path = drive_path + drive_destino_diagramas

renombrar_documentos_procesados = True #@param {type:"boolean"}

# levanta y procesa los documentos
process_FileNames = [ fn for fn in os.listdir( origen_dir_path ) if fn.endswith('.pdf') or fn.endswith('.docx') ]
print("\n> Documentos a procesar: ", len(process_FileNames))
for fn in process_FileNames:
  print("\n>", fn, ":")

  # determina nombre y tipo de archivo
  fnFull = origen_dir_path+"/"+fn
  nombre, tipo = os.path.splitext(fn)
  tipo = tipo.lower()

  if "pdf" in tipo:
    # procesa PDF
    res = fitz.open(fnFull)
    for i in range(len(res)):
      print("   Página ", i, "- imágenes encontradas: ", len(res.getPageImageList(i)))
      for img in res.getPageImageList(i):
          xref = img[0]
          pix = fitz.Pixmap(res, xref)
          imagFN = "/%s_P%sR%s.png" % (nombre, i, xref)
          print("\t"+imagFN, sep=' ')
          if pix.n < 5:       # this is GRAY or RGB
              pix.writePNG(destino_dir_path + imagFN)
          else:               # CMYK: convert to RGB first
              pix1 = fitz.Pixmap(fitz.csRGB, pix)
              pix1.writePNG(destino_dir_path + imagFN)
              pix1 = None
          pix = None
    # renombra archivo como procesado
    if renombrar_documentos_procesados:
      os.rename(fnFull, fnFull+"-LISTO")

  elif "doc" in tipo:    
      # procesa DOCx
      res = docx2python(fnFull)
      print("  imágenes encontradas: ", len(res.images.items()))      
      for iname, dimage in res.images.items():
        imagFN = "/%s_%s" % (nombre, iname)
        print("\t"+imagFN, sep=' ')
        imgFile = open(destino_dir_path + imagFN, 'wb')
        imgFile.write(dimage)
        imgFile.close()
      # renombra archivo como procesado
      if renombrar_documentos_procesados:
        os.rename(fnFull, fnFull+"-LISTO")
  else:
    print("Tipo de archivo desconocido: ", tipo)